# __Quality Checker__

In [18]:
import pandas as pd
import numpy as np
import re
import argparse
from PyPDF2 import PdfReader
import os
from textcleaningutils import *
import shutil as sh
from multiprocessing import Pool, Manager
import time
from tqdm.notebook import tqdm_notebook as tqdm

In [22]:
def qualityVerifier(filename):
    reader = PdfReader(f"Muestra2/{filename}")
    number_of_pages = len(reader.pages)
    doc_name = filename.replace("pdf", "txt")
    tcfinder = False
    for i in range(3):
        page = reader.pages[i]
        temp_text = page.extract_text()
        if "Table of Contents" in temp_text:
            tcfinder = True
            # tcnumber = i
            break
    table_contents = temp_text
    
    table_contents = [1 if x in table_contents else 0 for x in ["Call Participants", "Presentation", "Question and Answer"]]
    if tcfinder and table_contents[0] == 1 and table_contents[1] == 1:
        participants_num = int(temp_text.split("\n")[-3].replace(".", "")[-1]) - 1
        presentation_num = int(temp_text.split("\n")[-2].replace(".", "")[-1]) - 1
        qa_num = int(temp_text.split("\n")[-1].replace(".", "").split(" ")[-1]) - 1
        quality = 1
    elif tcfinder and table_contents[0] == 0 and table_contents[2] == 1:
        participants_num = 0
        presentation_num = int(temp_text.split("\n")[-2].replace(".", "")[-1]) - 1
        qa_num = int(temp_text.split("\n")[-1].replace(".", "").split(" ")[-1]) - 1
        quality = 2
    elif tcfinder and table_contents[0] == 0 and table_contents[2] == 0:
        participants_num = 0
        presentation_num = int(temp_text.split("\n")[-1].replace(".", "")[-1]) - 1
        qa_num = 0
        quality = 3
    elif not tcfinder:
        page = reader.pages[0]
        temp_text = page.extract_text()
    if "Executives" in temp_text:
        table_contents = [2, 2, 2]
        quality = 5
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Call Participants" in temp_text:
                participants_num = i
                break
            else:
                participants_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Presentation" in temp_text:
                presentation_num = i
                break
            else:
                presentation_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Question and Answer" in temp_text:
                qa_num = i
                break
            else:
                qa_num = 99
    elif "CORPORATE PARTICIPANTS" in reader.pages[1].extract_text():
        table_contents = [2, 2, 2]
        quality = 6
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Call Participants" in temp_text:
                participants_num = i
                break
            else:
                participants_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "PRESENTATION" in temp_text:
                presentation_num = i
                break
            else:
                presentation_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "QUESTIONS AND ANSWERS" in temp_text:
                qa_num = i
                break
            else:
                qa_num = 99
            
    elif "CONFERENCE CALL PARTICIPANTS" in reader.pages[1].extract_text():
        table_contents = [2, 2, 2]
        quality = 7
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "CONFERENCE CALL PARTICIPANTS" in temp_text:
                participants_num = i
                break
            else:
                participants_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "PRESENTATION" in temp_text:
                presentation_num = i
                break
            else:
                presentation_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "QUESTIONS AND ANSWERS" in temp_text:
                qa_num = i
                break
            else:
                qa_num = 99 
    else:
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Call Participants" in temp_text:
                participants_num = i
                break
            else:
                participants_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Presentation" in temp_text:
                presentation_num = i
                break
            else:
                presentation_num = 99
        for i in range(number_of_pages):
            page = reader.pages[i]
            temp_text = page.extract_text()
            if "Question and Answer" in temp_text:
                qa_num = i
                break
            else:
                qa_num = 99
        quality = 4
        table_contents = [3, 3, 3]

    return quality

In [23]:
badfiles = os.listdir("Muestra2")

In [25]:
qualityoutput = []
for i in tqdm(badfiles):
    try: 
        d = {
            "filename" : i ,
            "quality" : qualityVerifier(i)
        }
        qualityoutput.append(d)
    except:
        print(f"error with file {i}")

  0%|          | 0/1000 [00:00<?, ?it/s]

error with file 2017-May-05-KMP_u.TO-139842476726-transcript.pdf
error with file 2002-Oct-24-MRK.N-139745478137-transcript.pdf
error with file 2017-Jun-08-TRZ.TO-138218116979-transcript.pdf
error with file 2017-Feb-22-WCN.TO-137779077536-transcript.pdf
error with file 2017-Mar-02-MUX.N-136929750925-transcript.pdf


In [26]:
df = pd.DataFrame(qualityoutput)

In [27]:
df.value_counts("quality")

quality
6    989
7      5
4      1
Name: count, dtype: int64